# 행정동 단위의 그림

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import csv

import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px
import folium
# import networkx as nx

In [2]:
with open(
        "../참고 데이터/csv/large-datasets/2022/생활이동_행정동_202201/생활이동_행정동_2022.01_00시.csv",
        encoding="cp949") as f:
    # data = csv.reader(f)
    data = pd.read_csv(f)

print(data)
# data.sort_values(by='평균 이동 시간(분)')[::-1].head(20)
# for row in data:
# print(row)


           대상연월 요일  도착시간  출발 행정동 코드  도착 행정동 코드 성별  나이 이동유형  평균 이동 시간(분)  \
0        202201  일     0    1101053    1101053  F  15   HH           10   
1        202201  일     0    1101053    1101053  F  15   WH           10   
2        202201  일     0    1101053    1101053  F  20   EH           10   
3        202201  일     0    1101053    1101053  F  25   EE           10   
4        202201  일     0    1101053    1101053  F  25   HH           10   
...         ... ..   ...        ...        ... ..  ..  ...          ...   
1647126  202201  토     0      39000    1125073  F  45   EH           70   
1647127  202201  토     0      39000    1125073  M  15   EH           80   
1647128  202201  토     0      39000    1125074  F  20   EE          100   
1647129  202201  토     0      39000    1125074  M  25   EE           80   
1647130  202201  토     0      39000    1125074  M  50   EH           70   

        이동인구(합)  
0          4.04  
1          4.04  
2             *  
3             *  
4        

In [3]:
with open("../참고 데이터/한국행정구역분류_행정동코드(서울).csv", encoding='cp949') as f:
    data1 = pd.read_csv(f)
    #newdata = data1[['소분류', '읍면동']]

In [14]:
data = data.drop(data[data['출발 행정동 코드'] == data['도착 행정동 코드']].index)
# data.loc[data['이동인구(합)'] == '*', '이동인구(합)'] = 0/
data['이동인구(합)'] = data['이동인구(합)'].replace(to_replace="*", value=0)
data['이동인구(합)'] = data['이동인구(합)'].astype(float)

data = data.drop(data[(data['출발 행정동 코드'] < 1101053)].index)
data = data.drop(data[(data['출발 행정동 코드'] > 1125074)].index)
data = data.drop(data[(data['도착 행정동 코드'] < 1101053)].index)
data = data.drop(data[(data['도착 행정동 코드'] > 1125074)].index)

print(data)
# tst = {key: value for key, value in zip(data1['소분류'], "서울특별시 " + data1['시군구']+" "+data1['읍면동'])} # 이건 자치구용
# print(tst)
# data = data.replace(tst)
#tst = {11010:"ㅁㄴㅇㄹ"}

print(data.dtypes)

           대상연월 요일  도착시간  출발 행정동 코드  도착 행정동 코드 성별  나이 이동유형  평균 이동 시간(분)  \
31       202201  일     0    1101053    1101055  F  25   WH           10   
32       202201  일     0    1101053    1101055  F  35   HE           10   
33       202201  일     0    1101053    1101055  M  35   EH           10   
34       202201  일     0    1101053    1101056  F  30   EH           10   
35       202201  일     0    1101053    1101056  F  30   WH           20   
...         ... ..   ...        ...        ... ..  ..  ...          ...   
1603640  202201  토     0    1125074    1125073  M  45   WH           10   
1603641  202201  토     0    1125074    1125073  M  60   EE           80   
1603642  202201  토     0    1125074    1125073  M  60   EW           10   
1603643  202201  토     0    1125074    1125073  M  60   WH           10   
1603644  202201  토     0    1125074    1125073  M  75   WW           10   

         이동인구(합)  
31          0.00  
32          6.05  
33          6.05  
34          0.00  
35  

In [18]:
# data2 = data.groupby(['출발 시군구 코드', '도착 시군구 코드'])['평균 이동 시간(분)'].sum().sort_values()#.head(10)
# print(data2)

# data['이동인구(합)'] = data[].astype(float)

# data2point5 = pd.DataFrame(data.groupby(['도착 시군구 코드'])['평균 이동 시간(분)'].sum().sort_values())
data2point5 = pd.DataFrame(data.groupby(['도착 행정동 코드'])['이동인구(합)'].sum())
data2point5 = data2point5.reset_index()
# data2point5['도착 행정동 코드'] = str(data2point5['도착 행정동 코드'])
# data2point5.sort_values('도착 행정동 코드')
data2point5['도착 행정동 코드'] = data2point5['도착 행정동 코드'].astype(str)
data2point5.dtypes

도착 행정동 코드     object
이동인구(합)      float64
dtype: object

In [19]:
import json

with open('./geojson/서울_행정동_경계_2017.geojson') as j:
    seoul_geo = json.load(j)

m = folium.Map(location=[37.559819, 126.963895],
               zoom_start=11,
               tiles='cartodbpositron')

folium.Choropleth(
    geo_data=seoul_geo,
    name="choropleth",
    data=data2point5,
    columns=["도착 행정동 코드", "이동인구(합)"],
    key_on="properties.adm_cd",
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=.1,
    legend_name="레게노",
).add_to(m)

folium.LayerControl().add_to(m)

m.save("../작업 결과물/지도지도.html")

m

# 합친거

In [ ]:
import json

with open(
        "../참고 데이터/csv/large-datasets/2022/생활이동_행정동_202201/생활이동_행정동_2022.01_00시.csv",
        encoding="cp949") as f:
    # data = csv.reader(f)
    data = pd.read_csv(f)

data = data.drop(data[data['출발 행정동 코드'] == data['도착 행정동 코드']].index)
# data.loc[data['이동인구(합)'] == '*', '이동인구(합)'] = 0/
data['이동인구(합)'] = data['이동인구(합)'].replace(to_replace="*", value=0)
data['이동인구(합)'] = data['이동인구(합)'].astype(float)

data = data.drop(data[(data['출발 행정동 코드'] < 1101053)].index)
data = data.drop(data[(data['출발 행정동 코드'] > 1125074)].index)
data = data.drop(data[(data['도착 행정동 코드'] < 1101053)].index)
data = data.drop(data[(data['도착 행정동 코드'] > 1125074)].index)

data2point5 = pd.DataFrame(data.groupby(['도착 행정동 코드'])['이동인구(합)'].sum())
data2point5 = data2point5.reset_index()
data2point5['도착 행정동 코드'] = data2point5['도착 행정동 코드'].astype(str)

m = folium.Map(location=[37.559819, 126.963895],
               zoom_start=11,
               tiles='cartodbpositron')

folium.Choropleth(
    geo_data=seoul_geo,
    name="choropleth",
    data=data2point5,
    columns=["도착 행정동 코드", "이동인구(합)"],
    key_on="properties.adm_cd",
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=.1,
    legend_name="레게노",
).add_to(m)

folium.LayerControl().add_to(m)

m.save("../작업 결과물/지도지도.html")

m